### How to use the compiler folder for the example of LeNet-5

**Introduction to LeNet-5 : one of the first CNNs, useful for image recognition.**

*Architecture :* 
- **Input image :** 32x32 pixels, 1 channel
- **First convolutional layer C1 :** 6 convolutional filters of size 5x5, resulting in 6 feature maps of size 28x28 | ReLU activation
- **Average pooling layer AP2 :** 2x2 kernel with stride = 1 (ou 2?), resulting in feature maps of size 14x14 

*How to use data_definition :*
**Toolbox :** `numpy` tool is required for matrices manipulation : `conda install numpy` for a conda environment

**Defining the size of the matrices :** 
The input image, represented by an input tensor matrix of size 32x32x1 (height x width x channels), goes through C1 to become an output tensor of size 28x28x6.
The tensor is then converted, to obtain 2D matrices (Im2row) : an input matrix A (784x25) and a weight matrix B (25x6), whose multiplication results in an output matrix of size 784x6. This is done by ACETONE. Randomized matrices are used in their stead for the rest of the convolutional process.
The dimensions of the matrices are obtained using `tensor_matrix_converter.py` (no matrices are generated, only the dimensions):

In [4]:
%run ../compiler/data_definition/tensor_matrix_converter.py


Input tensor: nc = 1, nh = 32, nw = 32 
Output tensor: mc = 6, mh = 28, mw = 28 
Kernel: fh = 5, fw = 5 
Parameters: stride = (1, 1), pad = (0, 0) 


Input matrix: height = 784, width = 25 
Weight matrix: height = 25, width = 6 
Output matrix: height = 784, width = 6 




**Configuring the data generation :** 
i.e. whether to randomize the content of the matrices, to pad them, to use an activation function or not (ReLU), what type of files to write / print (JSON, binary), etc...
For that, `user_configuration.py` is to be used (adjusting the parameters to True / False depending on the desired outcome).
*(More details regarding LeNet-5 architech and parameter choices? See README)*

*For example, these parameters initialise the 784x25 input matrix A and 25x6 weight matrix B, so that their content is randomized.*

isInitRandom = True
A_row = 784
A_col = 25
B_col = 6

*As the VTA requires square 16x16 matrices for multiplication ; a ReLU activation is then used :*

block_size = 16
isSquare = True
useReLU = True

*We want JSON files as outputs, so :*

doWriteBinaryFile = False
doWriteJSON = True

In [5]:
%run ../compiler/data_definition/user_configuration.py

**Generating the data :**
If configured to obtain a JSON file (to be used for simulation), the program `main_matrix_generator.py` will generate a .json file in the ../OUTPUT/ folder.
It calls functions from several other programs : 
- `matrix_generator.py` : is used to generate the input and weight matrices (A size 784x25 and B size 25x6), according to `user_configuration.py` : the number of rows (height) and columns (width) of the matrix, the padding, if its content is to be randomized or filled with 0s. A and B are to be padded into 784x32 and 32x16 matrices for ease of splitting.
- `matrix_split.py` : needed to split A and B into square 16x16 sub-matrices, as is required by the VTA (only takes matrices of this size for matrix multiplications).
- `matrix_multiplication.py` : used for block matrix multiplication. A_block_i (16x16) and B_block_j (16x16) are multiplied to obtain an output sub-matrix (size 16x16 also). If the function ReLU is used, it also applies that to each of the values in the output matrices.
- `json_generator.py` : rather than outputting a binary file, hexadecimal instructions are explicitly given to generate a .json file, where the data from the output matrices are translated.

Using the example of LeNet-5 first convolutional layer C1 :

In [9]:
import numpy as np

# Example :
# Initialising with 48x25 A input matrix and 25x6 B weight matrix
A_row = 48
A_col = 25
B_col = 6

input_matrix = np.random.randint(1, A_col*A_row, size=(A_row, A_col))
weight_matrix = np.random.randint(1, A_col*B_col, size=(A_col,B_col))
print("Input Matrix (",A_row, "x", A_col,") :\n", input_matrix)
print("Weight Matrix (",A_col, "x", B_col,") :\n", weight_matrix)

Padded Input Matrix ( 48 x 32 ) :
 [[810 889  56 ...   0   0   0]
 [874 113 224 ...   0   0   0]
 [193 894 614 ...   0   0   0]
 ...
 [714 729  29 ...   0   0   0]
 [950 579 255 ...   0   0   0]
 [363 245 468 ...   0   0   0]]
Padded Weight Matrix ( 32 x 16 ) :
 [[ 11  71  86  66  20  52   0   0   0   0   0   0   0   0   0   0]
 [ 62  24 111  94 132 109   0   0   0   0   0   0   0   0   0   0]
 [139  50  44  41  43 142   0   0   0   0   0   0   0   0   0   0]
 [106  87  31 130  38  98   0   0   0   0   0   0   0   0   0   0]
 [127 136 114 123  22   6   0   0   0   0   0   0   0   0   0   0]
 [116  42  89  44  72 145   0   0   0   0   0   0   0   0   0   0]
 [103 120 124  40   2  98   0   0   0   0   0   0   0   0   0   0]
 [  6  88  42  94  41   2   0   0   0   0   0   0   0   0   0   0]
 [ 52  27  42 142  23  69   0   0   0   0   0   0   0   0   0   0]
 [ 16  58  45  38 103 122   0   0   0   0   0   0   0   0   0   0]
 [128  59  48 104 100 103   0   0   0   0   0   0   0   0   0   0]


In [10]:
# Padding the matrices so their dimensions can be divided by 16
import sys
sys.path.append('../compiler/data_definition')
import matrix_generator

input_matrix_padded = matrix_generator.matrix_padding(input_matrix)
weight_matrix_padded = matrix_generator.matrix_padding(weight_matrix)
print("Padded Input Matrix (",input_matrix_padded.shape[0], "x", input_matrix_padded.shape[1],") :\n", input_matrix_padded)
print("Padded Weight Matrix (",weight_matrix_padded.shape[0], "x", weight_matrix_padded.shape[1],") :\n", weight_matrix_padded)

Padded Input Matrix ( 48 x 32 ) :
 [[810 889  56 ...   0   0   0]
 [874 113 224 ...   0   0   0]
 [193 894 614 ...   0   0   0]
 ...
 [714 729  29 ...   0   0   0]
 [950 579 255 ...   0   0   0]
 [363 245 468 ...   0   0   0]]
Padded Weight Matrix ( 32 x 16 ) :
 [[ 11  71  86  66  20  52   0   0   0   0   0   0   0   0   0   0]
 [ 62  24 111  94 132 109   0   0   0   0   0   0   0   0   0   0]
 [139  50  44  41  43 142   0   0   0   0   0   0   0   0   0   0]
 [106  87  31 130  38  98   0   0   0   0   0   0   0   0   0   0]
 [127 136 114 123  22   6   0   0   0   0   0   0   0   0   0   0]
 [116  42  89  44  72 145   0   0   0   0   0   0   0   0   0   0]
 [103 120 124  40   2  98   0   0   0   0   0   0   0   0   0   0]
 [  6  88  42  94  41   2   0   0   0   0   0   0   0   0   0   0]
 [ 52  27  42 142  23  69   0   0   0   0   0   0   0   0   0   0]
 [ 16  58  45  38 103 122   0   0   0   0   0   0   0   0   0   0]
 [128  59  48 104 100 103   0   0   0   0   0   0   0   0   0   0]


In [12]:
# Splitting the matrices into 16 x 16 matrices and displaying the first block (for each matrix A & B) that would be obtained using `matrix_split.py`
import matrix_split

block_input_matrix, input_block_col = matrix_split.matrix_splitting(input_matrix_padded)
block_weight_matrix, weight_block_col = matrix_split.matrix_splitting(weight_matrix_padded)
print("First Block Input Matrix (",block_input_matrix[0].shape[0], "x", block_input_matrix[0].shape[1],") :\n", block_input_matrix[0])
print("First Block Weight Matrix (",block_weight_matrix[0].shape[0], "x", block_weight_matrix[0].shape[1],") :\n", block_weight_matrix[0])

First Block Input Matrix ( 16 x 16 ) :
 [[ 810  889   56  137  965   67 1160 1078 1080 1192  771  572  924   25
   925 1100]
 [ 874  113  224  336  808  428 1124  686   11   26  511  891  327  952
   286  648]
 [ 193  894  614  445  279   86  810  685  360 1157  409 1090  348  865
   429 1174]
 [ 326  187  579 1118  501  838  841  694  915  334 1162  436 1135  334
    31  317]
 [ 751  932  478  492   48 1016  355  215 1057  649  395  705  762  759
   933  675]
 [ 249  874  530  105  723   41  124  500  112  700  848  479  532  894
   371  504]
 [   4  393 1128  884 1105  897  233  402  793   31  405  638  934  793
   518  722]
 [ 300  558 1178  556  679 1139  940  468  524  136  680  935  388  511
   427  873]
 [  39    1 1052  197  234  271  553  157   34  819  489  230 1111 1146
   903  345]
 [ 903   14  645  668  998  486  395  332  489  116  683  897  376  438
   929   57]
 [ 948  911  607  339  940 1059  880  410  367  535 1057  834  979  609
  1121 1162]
 [ 296  923  996 1176 101

In [13]:
# Those two block matrices we've just obtained are then multiplied using the VTA
import matrix_multiplication

block_output_matrix, combinations = matrix_multiplication.block_matrix_multiply(block_input_matrix, block_weight_matrix, input_block_col, weight_block_col)
print("First Block Output Matrix (",block_output_matrix[0].shape[0], "x", block_output_matrix[0].shape[1],") :\n", block_output_matrix[0])

First Block Output Matrix ( 16 x 16 ) :
 [[ 12598 -16798  32614  13013  14501  -8542      0      0      0      0
       0      0      0      0      0      0]
 [ 30688  22129   6191 -11028 -27106  -5413      0      0      0      0
       0      0      0      0      0      0]
 [-13921 -17811  31802 -21072 -28928  -3891      0      0      0      0
       0      0      0      0      0      0]
 [-19264  -1677    388 -23733  26019 -25401      0      0      0      0
       0      0      0      0      0      0]
 [-21612  28568  21731  31113 -16719  21568      0      0      0      0
       0      0      0      0      0      0]
 [-25964  -8551 -21622 -17716  25617  11136      0      0      0      0
       0      0      0      0      0      0]
 [   842  16606  -3296  -7712   4331 -26399      0      0      0      0
       0      0      0      0      0      0]
 [  9697 -17403  10278   3125  -7136  14356      0      0      0      0
       0      0      0      0      0      0]
 [ 14502 -15019 -14412 

In [6]:
# We can then input the data into a .json file

%run ../compiler/data_definition/main_matrix_generator.py examples.data_lenet5_conv1

Binary files successfully generated.
JSON file successfully generated.

 INITIAL MATRICES:
A_matrix: ((h, w) = (784, 25)) 
 [[-1  2 -4 ...  2  2 -1]
 [ 1 -1 -2 ...  2 -4  1]
 [ 2 -4  2 ...  0 -2  1]
 ...
 [ 0 -2 -1 ...  2 -1  1]
 [-2  1 -2 ...  2  2 -1]
 [ 0  2  0 ...  2 -1  0]]

 x 
 B_matrix: ((h, w) = (25, 6)) 
 [[-3 -1 -3 -4 -4 -2]
 [-2  0  1  2  2  2]
 [ 2 -2  1 -4  2 -1]
 [ 0 -2  0  0 -3 -4]
 [-4  2 -1 -1  1 -3]
 [ 1 -4 -3  0  0 -2]
 [ 1 -2 -4  1  1 -1]
 [-1  1  0  2  1 -4]
 [ 2  1 -3  0 -2  2]
 [-3 -1  1  1 -1 -3]
 [-2 -1  1 -3  0  0]
 [ 2 -1  1 -2  2 -3]
 [-2  0 -1 -1 -3  2]
 [ 2 -1  2  0  1 -4]
 [-1  2 -1 -4  1 -2]
 [ 1  0  0 -1 -4 -4]
 [-4 -4  0  1 -1  2]
 [-4  1 -3 -1 -2  2]
 [ 0 -4 -4 -1  0  0]
 [-1  2 -4  2  0 -4]
 [ 0 -4 -1  1 -1  1]
 [ 0  2 -2  0 -3 -1]
 [-4 -2 -2 -2  2 -4]
 [-1 -1  2 -3 -2 -4]
 [-1  0  0 -4  2  0]]


 X_matrix: ((h, w) = (1, 1)) 
 [[-3]]



 PADDED MATRICES:
A_padded: ((h, w) = (784, 32)) 
 [[-1  2 -4 ...  0  0  0]
 [ 1 -1 -2 ...  0  0  0]
 [ 2 -4  2 ..